In [66]:
import pandas as pd
import numpy as np
import datetime as dt

In [67]:
#Uniform dictionary of country names and corresponding ISOs
iso_df = pd.read_csv('../../data/raw/iso_3digit_alpha_country_codes.csv', low_memory=False)
rename_dict = iso_df.set_index('Definition').to_dict()['Code_Value']

#Import list of airsavvi countries
airsavvi = pd.read_csv('../../data/tidy/airsavvi_list.csv')
airsavvi['iso'] = airsavvi['country'].apply(lambda x: rename_dict.get(x))

#Mismatch between 3 digit ISO country names and Airsavvi list of countries
airsavvi.loc[airsavvi.country == 'Russia', 'iso'] = "RUS"
airsavvi.loc[airsavvi.country == 'Serbia', 'iso'] = "SRB"
airsavvi.loc[airsavvi.country == 'Taiwan', 'iso'] = "TWN"
airsavvi.loc[airsavvi.country == 'Republic of Korea', 'iso'] = "KOR"
airsavvi.loc[airsavvi.country == 'Czech Republic', 'iso'] = "CZE"

In [68]:
df = pd.read_csv('../../data/raw/govt_measures.csv')
df = df[['ISO','CATEGORY','ENTRY_DATE']]
df.ENTRY_DATE = pd.to_datetime(df.ENTRY_DATE)

#Subset of countries with airsavvi flight data from Feb 15 to Aug 31
df = df.loc[df['ISO'].isin(airsavvi['iso'])]
df.head()
#df.CATEGORY.str.replace('Public health measures', 'PH', regex=False)
# PH = public health
# SE = govt and socio-economic
# SD = social distancing
# MR = movement restrictions
# LD = lockdown
# HE = humanitarian exemption

,ISO,CATEGORY,ENTRY_DATE
57,ALB,Movement restrictions,2020-03-14
58,ALB,Movement restrictions,2020-03-14
59,ALB,Social distancing,2020-03-14
60,ALB,Social distancing,2020-03-14
61,ALB,Movement restrictions,2020-03-14


In [69]:
start_date = dt.datetime(2020,2,15) #.datetime.now().date())
date_range = pd.date_range(start_date, periods=199, freq='D')

#Set MultiIndex name
idx = pd.MultiIndex.from_product([df.ISO.unique(), df.CATEGORY.unique()])
idx.set_names(['iso', 'category'], inplace=True)

idf = pd.DataFrame(
    data=0,
    index=idx, #later, maybe: ['PH','SE','MR','LD','HE'] instead of actual category names
    columns=date_range
)

#Subset of countries with airsavvi flight data from Feb 15 to Aug 31
idf = idf.loc[idf.index.get_level_values('iso').isin(airsavvi['iso'])] #get_level_values for accessing MultiIndex

idf.head()

2020-02-15  2020-02-16  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-17  2020-02-18  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-19  2020-02-20  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-21  2020-02-22  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-23  2020-02-24  ...  \
iso category                                                        ...   
ALB Movement restrictions                            0           0  ...   
    Social distancing                                0           0  ...   
    Public health measures                           0           0  ...   
    Governance and socio-economic measures           0           0  ...   
    Lockdown                                         0           0  ...   

                                            2020-08-22  2020-08-23  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-24  2020-08-25  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-26  2020-08-27  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-28  2020-08-29  \
iso categor

In [70]:
for i in df.index:
    mdate = df.loc[i].ENTRY_DATE
    if (mdate > dt.datetime(2020,2,14)) & (mdate <= dt.datetime(2020, 9, 1)):
        idf.loc[(df.loc[i].ISO, df.loc[i].CATEGORY),df.loc[i].ENTRY_DATE] = 1

In [71]:
idf.loc[('ALB', 'Public health measures'),'2020-03-14']

1

In [72]:
#Print output in tidy
idf.to_csv('../../data/tidy/govt_interventions.csv')
idf.head()

2020-02-15  2020-02-16  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-17  2020-02-18  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-19  2020-02-20  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-21  2020-02-22  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-02-23  2020-02-24  ...  \
iso category                                                        ...   
ALB Movement restrictions                            0           0  ...   
    Social distancing                                0           0  ...   
    Public health measures                           0           0  ...   
    Governance and socio-economic measures           0           0  ...   
    Lockdown                                         0           0  ...   

                                            2020-08-22  2020-08-23  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-24  2020-08-25  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-26  2020-08-27  \
iso category                                                         
ALB Movement restrictions                            0           0   
    Social distancing                                0           0   
    Public health measures                           0           0   
    Governance and socio-economic measures           0           0   
    Lockdown                                         0           0   

                                            2020-08-28  2020-08-29  \
iso categor

In [73]:
# Examples
idf.loc[idf.index.get_level_values('category')=='Movement restrictions']

,,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,...,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31
iso,category,,,,,,,,,,,,,,,,,,,,,
ALB,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ARG,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AUS,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AUT,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
BEL,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ARE,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
GBR,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
USA,Movement restrictions,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
